In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(15087, 128, 128, 3)

In [8]:
y.shape

(15087, 8)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(12069, 128, 128, 3)

In [11]:
X_test.shape

(3018, 128, 128, 3)

In [12]:
y_train.shape

(12069, 8)

In [13]:
y_test.shape

(3018, 8)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

8

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 12:01:37.448280: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


378/378 [==============================] - 81s 212ms/step - loss: 0.5540 - accuracy: 0.8033 - val_loss: 0.7150 - val_accuracy: 0.8042
Epoch 2/10
378/378 [==============================] - 86s 228ms/step - loss: 0.2439 - accuracy: 0.9132 - val_loss: 0.1906 - val_accuracy: 0.9235
Epoch 3/10
378/378 [==============================] - 76s 201ms/step - loss: 0.1590 - accuracy: 0.9458 - val_loss: 0.0987 - val_accuracy: 0.9652
Epoch 4/10
378/378 [==============================] - 89s 235ms/step - loss: 0.1395 - accuracy: 0.9526 - val_loss: 0.1943 - val_accuracy: 0.9294
Epoch 5/10
378/378 [==============================] - 91s 240ms/step - loss: 0.1022 - accuracy: 0.9649 - val_loss: 0.1205 - val_accuracy: 0.9563
Epoch 6/10
378/378 [==============================] - 91s 240ms/step - loss: 0.1067 - accuracy: 0.9621 - val_loss: 0.0662 - val_accuracy: 0.9778
Epoch 7/10
378/378 [==============================] - 85s 224ms/step - loss: 0.0900 - accuracy: 0.9704 - val_loss: 0.0876 - val_accuracy: 0.9

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 97.91%


In [19]:
cnn_scores

[0.060293544083833694, 0.9791252613067627]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

95/95 [==============================] - 10s 104ms/step
Accuracy: 97.91%
Precision: 97.93%
Recall: 97.91%
F1 score: 97.91%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-apel+jagung.csv', index=False)

+------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                      | True Labels                                        | Predicted Labels                                   |
|------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|    0 | daf1c801-8722-44d6-9f3f-be0250d952e6___RS_NLB 3941_flipTB.JPG                 | Corn_(maize)___Northern_Leaf_Blight                | Corn_(maize)___Northern_Leaf_Blight                |
|    1 | 39facd4b-a088-4d14-8261-5e54beabc108___RS_NLB 4201_180deg.JPG                 | Corn_(maize)___Northern_Leaf_Blight                | Corn_(maize)___Northern_Leaf_Blight                |
|    2 | 2788c3da-4e36-40

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-apel+jagung.csv', index=False)

False Predictions:
+------+--------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                 | True Labels                                        | Predicted Labels                                   |
|------+--------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|   83 | 05f92471-3cd4-441b-af21-1a02304d0b6c___RS_GLSp 7315.JPG                  | Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot | Corn_(maize)___Northern_Leaf_Blight                |
|   94 | 36584bae-f3bb-40c0-b866-ed5ba0c2f7ae___RS_GLSp 4577.JPG                  | Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot | Corn_(maize)___Northern_Leaf_Blight                |
|  127 | 26a31865-1d42-4bd6-8b6

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

378/378 [==============================] - 44s 117ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 843.8398621082306 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

95/95 [==============================] - 18s 191ms/step
Random Forest Classifier Accuracy: 98.18%
Random Forest Classifier Precision: 0.98
Random Forest Classifier Recall: 0.98
Random Forest Classifier F1 Score: 0.98


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-apel+jagung.csv', index=False)

+------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                      | True Labels                                        | Predicted Labels                                   |
|------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|    0 | daf1c801-8722-44d6-9f3f-be0250d952e6___RS_NLB 3941_flipTB.JPG                 | Corn_(maize)___Northern_Leaf_Blight                | Corn_(maize)___Northern_Leaf_Blight                |
|    1 | 39facd4b-a088-4d14-8261-5e54beabc108___RS_NLB 4201_180deg.JPG                 | Corn_(maize)___Northern_Leaf_Blight                | Corn_(maize)___Northern_Leaf_Blight                |
|    2 | 2788c3da-4e36-40

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-apel+jagung.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                             | True Labels                                        | Predicted Labels                                   |
|------+----------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|   83 | 05f92471-3cd4-441b-af21-1a02304d0b6c___RS_GLSp 7315.JPG              | Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot | Corn_(maize)___Northern_Leaf_Blight                |
|   94 | 36584bae-f3bb-40c0-b866-ed5ba0c2f7ae___RS_GLSp 4577.JPG              | Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot | Corn_(maize)___Northern_Leaf_Blight                |
|   99 | 20046515-e844-4ad2-b3e0-38d2dabcef66___R.S